# Parsing Human Proteome query results containing site and region features
2.7.2022 
@mfpfox

### help links
* Sequence annotation (Features) <https://www.uniprot.org/help/sequence_annotation>
* api query <https://www.uniprot.org/help/api_queries>

---

**columns parsed/formatted by downloadUniprotDomains.py:**
## SITES
1. SubstrateBindingSite
2. ActiveSite
3. MetalBindSite
4. ModifiedResidue
5. Lipidation
6. Glycosylation
7. Mutagenesis
8. DisulfideBond

## REGIONS
0. topological domain (not included in saved output from script)
1. Domain
2. NucleotideBindRegion
3. DNAbindRegion
4. ZincFinger
5. Motif
6. DisorderedRegion
7. BetaStrand
8. Helix
9. Turn

---

## TODO:
- add section of code that checks if query result sequence is identical to ref of cpdaa and category positions
    - if sequence is not identical, are positions of cpdaa and category unchanged?
    - if sequence is same length but different, qc that site modifications match reference aa type at position
- **add substrings corresponding to zinc fingers, domains, other regions that may be of interest for aa abundance analysis**
- combine all functionality of will's code and brandon's code

**consider changing all partition code to (;) instead of (;  )**

---
---
---

# SITES

1. SubstrateBindingSite
2. ActiveSite
3. MetalBindSite
4. ModifiedResidue
5. Lipidation
6. Glycosylation
7. Mutagenesis
8. DisulfideBond

## parsing BINDING SITE, Binding site for any chemical group (co-enzyme, prosthetic group, etc.) 
**for Substrate binding only**
```python
binding = 'BINDING 277;  /note="Substrate; via carbonyl oxygen";  /evidence="ECO:0000269|PubMed:18272479, ECO:0000269|PubMed:26074468"; BINDING 301;  /note="strate";  /evidence="ECO:0000269|PubMed:18272479, ECO:0000269|PubMed:23159739, ECO:0000269|PubMed:26074468"; BINDING 339;  /note="Substrate";  /evidence="ECO:0000269|PubMed:18272479, ECO:0000269|PubMed:23159739, ECO:0000269|PubMed:26074468"; BINDING 341;  /note="Substrate";  /evidence="ECO:0000269|PubMed:18272479, ECO:0000269|PubMed:23159739, ECO:0000269|PubMed:26074468"; BINDING 497;  /note="Substrate";  /evidence="ECO:0000269|PubMed:18272479, ECO:0000269|PubMed:23159739, ECO:0000269|PubMed:26074468"; BINDING 501;  /note="Substrate";  /evidence="ECO:0000269|PubMed:18272479, ECO:0000269|PubMed:23159739, ECO:0000269|PubMed:26074468"'
```
**what parsing code does..**
* splits string on certain all caps word like BINDING, ZN_FING, STRAND, DOMAIN, etc.
* ignores positions prefaced by something other than all caps word used to split string and positions prefaced by  isoform id, ex P1234-3:2..4 ignored and TURN 10..14 ignored for BINDING split()
* only parses positions that are describing 'Substrate' binding site
* parses single position only and ignores 2 position annotation (checks for ..)
* outputs single positions as list

In [6]:
binding = 'BINDING P1234-4:277;  /note="Substrate; via carbonyl oxygen";  /evidence="ECO:0000269"; BINDING 301;  /note="strate";  /evidence="ECO:0000269"; BINDING 339;  /note="Substrate";  /evidence="ECO:0000269"; BINDING 341;  /note="Substrate";  /evidence="ECO:0000269|PubMed:18272479"; BINDING 497;  /note="Substrate";  /evidence="ECO:0000269|PubMed:18272479, ECO:0000269|PubMed:23159739, ECO:0000269|PubMed:26074468"; BINDING 501;  /note="Substrate";  /evidence="ECO:0000269|PubMed:18272479, ECO:0000269|PubMed:23159739, ECO:0000269|PubMed:26074468"'
binding = binding.split("BINDING ")
bind_coords=[]
for item in binding:
    if "Substrate" in item:
        # removed two white spaces in partition in case there is no note or evidence
        # replaced split("..") with strip() to remove whitespaces
        bind_coords.append(item.partition(";")[0].strip())
bind_coords2 = []
for item in bind_coords:
    if len(item) > 1:
        # checks if there is isoform id preface or if range for other type of annotation
        if "-" not in item and " " not in item and ".." not in item:
            bind_coords2.append(item)
    else:
        continue

#print(binding)
print(bind_coords2)

['339', '341', '497', '501']


## parsing ACTIVE SITE, Amino acid(s) directly involved in the activity of an enzyme 
```python
active = 'ACT_SITE 99;  /evidence="ECO:0000250"; ACT_SITE 170;  /evidence="ECO:0000250"'
active = 'ACT_SITE 99;  /note="Proton acceptor";  /evidence="ECO:0000269|PubMed:11988089, ECO:0000269|PubMed:14573603"'
active = 'ACT_SITE 99;  /note="Proton donor/acceptor";  /evidence="ECO:0000305|PubMed:23481380"'
```
**what parsing code does..**
* splits string on certain all caps word like ACT_SITE, ZN_FING, STRAND, DOMAIN, etc.
* ignores positions prefaced by something other than all caps word used to split string and positions prefaced by  isoform id, ex P1234-3:2..4 ignored and TURN 10..14 ignored for ACT_SITE split()
* parses single position only and ignores 2 position annotation (checks for ..)
* outputs single positions as list

In [149]:
active = 'ACT_SITE 99;  /note="Proton acceptor";  /evidence="ECO:0000269|PubMed:11988089, ECO:0000269|PubMed:14573603"'
active = active.split("ACT_SITE ")
active_coords=[]
for item in active:
    # removed two white spaces in partition in case there is no note or evidence
    # replaced split("..") with strip() to remove whitespaces
    active_coords.append(item.partition(";")[0].strip()) 
active_coords2 = []
for item in active_coords:
    if len(item) > 1:
        # checks if there is isoform id preface or if range for other type of annotation
        if "-" not in item and " " not in item and ".." not in item:
            active_coords2.append(item)
    else:
        continue


print(active)
print(active_coords2)

['', '99;  /note="Proton acceptor";  /evidence="ECO:0000269|PubMed:11988089, ECO:0000269|PubMed:14573603"']
['99']


## parsing METAL BINDING SITE, Binding site for a metal ion 
```python
metal = 'METAL 10;  /note="Magnesium"; METAL 12;  /note="Magnesium; via carbonyl oxygen"; METAL 145;  /note="Magnesium"'
```
**what parsing code does..**
* splits string on certain all caps word like METAL, ZN_FING, STRAND, DOMAIN, etc.
* ignores positions prefaced by something other than all caps word used to split string and positions prefaced by  isoform id, ex P1234-3:2..4 ignored and TURN 10..14 ignored for METAL split()
* parses single position only and ignores 2 position annotation (checks for ..)
* outputs single positions as list

In [151]:
metal = 'METAL 10;  /note="Magnesium"; METAL 12;  /note="Magnesium; via carbonyl oxygen"; METAL 145;  /note="Magnesium"'
metal = metal.split("METAL ")
metal_coords=[]
for item in metal:
    # removed two white spaces in partition in case there is no note or evidence
    # replaced split("..") with strip() to remove whitespaces
    metal_coords.append(item.partition(";")[0].strip()) 
metal_coords2 = []
for item in metal_coords:
    if len(item) > 1:
        # checks if there is isoform id preface or if range for other type of annotation
        if "-" not in item and " " not in item and ".." not in item:
            metal_coords2.append(item)
    else:
        continue


print(metal_coords)
print(metal_coords2)

['', '10', '12', '145']
['10', '12', '145']


## parsing MODIFIED RESIDUE, Modified residues excluding lipids, glycans and protein cross-links 
checking browser format of feature <>
```python
modresidue = 'MOD_RES 996;  /note="Phosphoserine";  /evidence="ECO:0007744|PubMed:19690332, ECO:0007744|PubMed:20068231, ECO:0007744|PubMed:21406692, ECO:0007744|PubMed:23186163"; MOD_RES 998;  /note="Phosphothreonine";  /evidence="ECO:0007744|PubMed:20068231, ECO:0007744|PubMed:21406692, ECO:0007744|PubMed:23186163"'
```
**what parsing code does..**
* splits string on certain all caps word like MOD_RES, ZN_FING, STRAND, DOMAIN, etc.
* ignores positions prefaced by something other than all caps word used to split string and positions prefaced by  isoform id, ex P1234-3:2..4 ignored and TURN 10..14 ignored for MOD_RES split()
* parses single position only and ignores 2 position annotation (checks for ..)
* outputs single positions as list

In [153]:
modresidue = 'MOD_RES 996;  /note="Phosphoserine";  /evidence="ECO:0007744|PubMed:19690332, ECO:0007744|PubMed:20068231, ECO:0007744|PubMed:21406692, ECO:0007744|PubMed:23186163"; MOD_RES 998;  /note="Phosphothreonine";  /evidence="ECO:0007744|PubMed:20068231, ECO:0007744|PubMed:21406692, ECO:0007744|PubMed:23186163"'
modresidue = modresidue.split("MOD_RES ")
mod_coords=[]
for item in modresidue:
    # removed two white spaces in partition in case there is no note or evidence
    # replaced split("..") with strip() to remove whitespaces
    mod_coords.append(item.partition(";")[0].strip()) 
mod_coords2 = []
for item in mod_coords:
    if len(item) > 1:
        # checks if there is isoform id preface or if range for other type of annotation
        if "-" not in item and " " not in item and ".." not in item:
            mod_coords2.append(item)
    else:
        continue


print(mod_coords)
print(mod_coords2)

['', '996', '998']
['996', '998']


## parsing LIPIDATION, Covalently attached lipid group(s) 
```python 
lipid = 'LIPID 10;  /note="S-palmitoyl cysteine";  /evidence="ECO:0000269|PubMed:15840583"; LIPID 11;  /note="S-palmitoyl cysteine";  /evidence="ECO:0000269|PubMed:15840583"; LIPID Q96GW7-2:646;  /note="GPI-anchor amidated alanine";  /evidence="ECO:0000255"'
```
**what parsing code does..**
* splits string on certain all caps word like LIPID, MOD_RES, ZN_FING, STRAND, DOMAIN, etc.
* ignores positions prefaced by something other than all caps word used to split string and positions prefaced by  isoform id, ex P1234-3:2..4 ignored and TURN 10..14 ignored for LIPID split()
* parses single position only and ignores 2 position annotation (checks for ..)
* outputs single positions as list

In [155]:
lipid = 'LIPD 10;  /note="S-palmitoyl cysteine";  /evidence="ECO:0000269|PubMed:15840583"; LIPID 11;  /note="S-palmitoyl cysteine";  /evidence="ECO:0000269|PubMed:15840583"; LIPID Q96GW7-2:646;  /note="GPI-anchor amidated alanine";  /evidence="ECO:0000255"'
lipid = lipid.split("LIPID ")
lipid_coords=[]
for item in lipid:
    # removed two white spaces in partition in case there is no note or evidence
    # replaced split("..") with strip() to remove whitespaces
    lipid_coords.append(item.partition(";")[0].strip()) 
lipid_coords2 = []
for item in lipid_coords:
    if len(item) > 1:
        # checks if there is isoform id preface or if range for other type of annotation
        if "-" not in item and " " not in item and ".." not in item:
            lipid_coords2.append(item)
    else:
        continue


print(lipid_coords)
print(lipid_coords2)

['LIPD 10', '11', 'Q96GW7-2:646']
['11']


## parsing GLYCOSYLATION, Covalently attached glycan group(s) 
```python
glycos = 'CARBOHYD Q9BXJ4-3:70;  /note="N-linked (GlcNAc...) asparagine";  /evidence="ECO:0000269|PubMed:7505568"; CARBOHYD 106;  /note="N-linked (GlcNAc...) asparagine; partial";  /evidence="ECO:0000269|PubMed:7505568"; CARBOHYD 223;  /note="N-linked (GlcNAc...) asparagine";  /evidence="ECO:0000269|PubMed:7505568"; CARBOHYD 246;  /note="N-linked (GlcNAc...) asparagine";  /evidence="ECO:0000269|PubMed:7505568"; CARBOHYD 315;  /note="N-linked (GlcNAc...) asparagine";  /evidence="ECO:0000269|PubMed:7505568"; CARBOHYD 406;  /note="N-linked (GlcNAc...) asparagine";  /evidence="ECO:0000269|PubMed:7505568"; CARBOHYD 450;  /note="N-linked (GlcNAc...) asparagine";  /evidence="ECO:0000269|PubMed:7505568"; CARBOHYD 454;  /note="N-linked (GlcNAc...) asparagine";  /evidence="ECO:0000269|PubMed:7505568"'
```

In [157]:
glycos = 'CARBOHYD Q9BXJ4-3:70;  /note="N-linked (GlcNAc...) asparagine";  /evidence="ECO:0000269|PubMed:7505568"; CARBOHYD 106;  /note="N-linked (GlcNAc...) asparagine; partial";  /evidence="ECO:0000269|PubMed:7505568"; CARBOHYD 223;  /note="N-linked (GlcNAc...) asparagine";  /evidence="ECO:0000269|PubMed:7505568"; CARBOHYD 246;  /note="N-linked (GlcNAc...) asparagine";  /evidence="ECO:0000269|PubMed:7505568"; CARBOHYD 315;  /note="N-linked (GlcNAc...) asparagine";  /evidence="ECO:0000269|PubMed:7505568"; CARBOHYD 406;  /note="N-linked (GlcNAc...) asparagine";  /evidence="ECO:0000269|PubMed:7505568"; CARBOHYD 450;  /note="N-linked (GlcNAc...) asparagine";  /evidence="ECO:0000269|PubMed:7505568"; CARBOHYD 454;  /note="N-linked (GlcNAc...) asparagine";  /evidence="ECO:0000269|PubMed:7505568"'
glycos = glycos.split("CARBOHYD ")
gly_coords=[]
for item in glycos:
    # removed two white spaces in partition in case there is no note or evidence
    # replaced split("..") with strip() to remove whitespaces
    gly_coords.append(item.partition(";")[0].strip()) 
gly_coords2 = []
for item in gly_coords:
    if len(item) > 1:
        # checks if there is isoform id preface or if range for other type of annotation
        if "-" not in item and " " not in item and ".." not in item:
            gly_coords2.append(item)
    else:
        continue


print(gly_coords)
print(gly_coords2)

['', 'Q9BXJ4-3:70', '106', '223', '246', '315', '406', '450', '454']
['106', '223', '246', '315', '406', '450', '454']


## parsing MUTAGENESIS, Site which has been experimentally altered by mutagenesis 
```python
mutagen = 'MUTAGEN 103..107;  /note="LLLDD->AAAAA: Reduces clathrin binding.";  /evidence="ECO:0000269|PubMed:11239472"; MUTAGEN 254;  /note="S->A: No effect on phosphorylation in vitro.";  /evidence="ECO:0000269|PubMed:12719431"; MUTAGEN 259;  /note="S->A,D,E: Abolishes phosphorylation, no change in activity.";  /evidence="ECO:0000269|PubMed:12719431"; MUTAGEN 259;  /note="S->A: Protects from proteolysis.";  /evidence="ECO:0000269|PubMed:12719431"; MUTAGEN 262;  /note="S->A: No effect on phosphorylation in vitro.";  /evidence="ECO:0000269|PubMed:12719431"; MUTAGEN 266;  /note="S->A: No effect on phosphorylation in vitro.";  /evidence="ECO:0000269|PubMed:12719431"; MUTAGEN 1250;  /note="D->A: Abolishes lipid kinase activity. Affects clathrin distribution when combined with a truncation encompassing the region of clathrin interaction.";  /evidence="ECO:0000269|PubMed:16215232"; MUTAGEN 1488;  /note="R->A: Reduces affinity for PtdIns(4,5)P2-containing membranes 7-fold.";  /evidence="ECO:0000269|PubMed:17038310"; MUTAGEN 1490;  /note="V->A: Reduces affinity for PtdIns(4,5)P2-containing membranes 7-fold.";  /evidence="ECO:0000269|PubMed:17038310"; MUTAGEN 1491;  /note="L->A: Reduces affinity for PtdIns(4,5)P2-containing membranes 5-fold.";  /evidence="ECO:0000269|PubMed:17038310"; MUTAGEN 1493;  /note="R->A: Reduces affinity for PtdIns(4,5)P2-containing membranes 23-fold.";  /evidence="ECO:0000269|PubMed:17038310"; MUTAGEN 1503;  /note="R->A: Abolishes interaction with PtdIns(4,5)P2-containing membranes.";  /evidence="ECO:0000269|PubMed:17038310"'

mutagen = 'MUTAGEN 1030;  /note="C->S: Abolishes enzymatic activity.";  /evidence="ECO:0000269|PubMed:16912044"![image.png](attachment:image.png)'
```

In [3]:
mutagen = 'MUTAGEN 99..1000;  /note="C->S: Abolishes enzymatic activity.";  /evidence="ECO:0000269|PubMed:16912044"![image.png](attachment:image.png); MUTAGEN 1030;MUTAGEN 33; '
mutagen = mutagen.split("MUTAGEN ")
mut_coords=[]
for item in mutagen:
    # removed two white spaces in partition in case there is no note or evidence
    # replaced split("..") with strip() to remove whitespaces
    mut_coords.append(item.partition(";")[0].strip()) 
mut_coords2 = []
for item in mut_coords:
    if len(item) > 1:
        # checks if there is isoform id preface or if range for other type of annotation
        if "-" not in item and " " not in item and ".." not in item:
            mut_coords2.append(item)
    else:
        continue


print(mut_coords)
print(mut_coords2)

['', '99..1000', '1030', '33']
['1030', '33']


## parsing DISULFIDE, Cysteine residues participating in disulfide bonds 
example of browser <https://www.uniprot.org/uniprot/P22303>
```python 
# for P22303 qc
disulfide ='DISULFID 100..127; DISULFID 288..303; DISULFID 440..560; DISULFID 611;  /note="Interchain"![image.png](attachment:image.png)'
disulfide = 'DISULFID 1156..1291;  /evidence="ECO:0000255|PROSITE-ProRule:PRU00580"; DISULFID 1178..1330;  /evidence="ECO:0000255|PROSITE-ProRule:PRU00580"; DISULFID 1542..1680;  /evidence="ECO:0000255|PROSITE-ProRule:PRU00580"; DISULFID 1564..1719;  /evidence="ECO:0000255|PROSITE-ProRule:PRU00580"; DISULFID 1931..2069;  /evidence="ECO:0000255|PROSITE-ProRule:PRU00580"; DISULFID 1953..2107;  /evidence="ECO:0000255|PROSITE-ProRule:PRU00580"; DISULFID 2331..2468;  /evidence="ECO:0000255|PROSITE-ProRule:PRU00580"; DISULFID 2712..2723;  /evidence="ECO:0000250"; DISULFID 2717..2732;  /evidence="ECO:0000250"; DISULFID 2734..2743;  /evidence="ECO:0000250"![image.png](attachment:image.png)'
```
**what parsing code does..**
* includes 2 position format (.. between values) and single position format
* ignores positions prefaced by something other than DISULFID and positions prefaced by uniprot isoform id, ex P1234-3:2..4 ignored and TURN 10..14 ignored
* ignores note and evidence text by not including items containing = chr
* output of 2 positions and single position separated by .., single position 611 output is 611..611

In [118]:
bonds ='D 100127; DISULFID 288..303; DISULFID 440..560; DISULFID 611;  /note="Interchain"![image.png](attachment:image.png)'
bond_coords2 = []
bonds = bonds.replace(" DISULFID ", "")
bonds = bonds.replace("DISULFID ", "")
bonds=bonds.split(";")
for item in bonds:
    # 2 positions
    if '..' in item:
        bond_coords = [item.split("..")[0], item.split("..")[1]]
        # checks if there is isoform id preface or if range for other type of annotation
        if "-" not in bond_coords[0] and " " not in bond_coords[0]:
            bond_coords2.append(bond_coords)
    # 1 position
    if '..' not in item:
        # checks if there is isoform id preface or if range for other type of annotation
        if "-" not in item and " " not in item and "=" not in item:
            v = []
            v.append(item)
            v.append(item)
            bond_coords2.append(v)
bridges = [x + ".." + y for x, y in bond_coords2]

print(bridges)

['288..303', '440..560', '611..611']


---
---
---
---

# REGIONS
0. topological domain (not included in saved output from script)
1. Domain
2. NucleotideBindRegion
3. DNAbindRegion
4. ZincFinger
5. Motif
6. DisorderedRegion
7. BetaStrand
8. Helix
9. Turn

## parsing TOPOLOGICAL DOMAIN, Location of non-membrane regions of membrane-spanning proteins 
**for Extracellular range parsing**

```python 
topological_domain="TOPO_DOM 24 118 Extracellular. {ECO:0000255}.; TOPO_DOM 118 Extracellular. {ECO:0000255}.; TOPO_DOM 139 139 Extracellular. {ECO:0000255}."
```
**what parsing code does..**
* splits string on certain all caps word like TOPO_DOM, ZN_FING, STRAND, DOMAIN, etc.
* ignores positions prefaced by something other than all caps word used to split string and positions prefaced by uniprot isoform id, ex P1234-3:2..4 ignored and TURN 10..14 ignored for TOPO_DOM split()
* only parses positions that are describing 'Extracellular' feature
* parses range format (includes .. between values) and single position annotation
* single position annotation converted to range format
* outputs 2 position range and single position as values separated by -

```python 
topological_domain="TOPODOM 1..149;  /note='Extracellular';  /evidence='ECO:0000305|PubMed:27999994'; TOPO_DOM 171;  /note='Extracellular';  /evidence='ECO:0000305'; TOPO_DOM 193..196;  /note='Cytoplasmic';  /evidence='ECO:0000305'; TOPO_DOM 218..220;  /note='Extracellular';  /evidence='ECO:0000305'; TOPO_DOM 242..259;  /note='Cytoplasmic';  /evidence='ECO:0000305'"                        
topo = topological_domain.split("TOPO_DOM ")
extracell_coords=[]
for item in topo:
    if "Extracellular" in item:
        extracell_coords.append(item.partition(";  ")[0].split(".."))
extracell_coords2 = []
for item in extracell_coords:
    # 2 position range
    if len(item) > 1:
        # checks if there is isoform id preface or if range for other type of annotation
        if "-" not in item[0] and " " not in item[0]:
            extracell_coords2.append(item)
    # single position
    elif len(item) == 1:
        # checks if there is isoform id preface or if range for other type of annotation
        if "-" not in item and " " not in item:
            v = []
            v.append(item[0])
            v.append(item[0])
            extracell_coords2.append(v)
    else:
        continue
extracell_ranges = [x + "-" + y for x, y in extracell_coords2]


print(extracell_ranges)
```

## parsing NUCLEOTIDE BINDING, Nucleotide phosphate binding region 
example on browser of feature <https://www.uniprot.org/uniprot/P49914>
```python 
nucleotide= 'NP_BIND 10..14;  /note="FAD";  /evidence="ECO:0000250|UniProtKB:Q9HFE4"; NP_BIND 41..42;  /note="FAD";  /evidence="ECO:0000250|UniProtKB:Q9HFE4"; NP_BIND 62..63;  /note="FAD";  /evidence="ECO:0000250|UniProtKB:Q9HFE4"; NP_BIND 196..199;  /note="NADP";  /evidence="ECO:0000250|UniProtKB:Q9HFE4"'
```
**what parsing code does..**
* splits string on certain all caps word like NP_BIND, ZN_FING, STRAND, DOMAIN, etc.
* ignores positions prefaced by something other than all caps word used to split string and positions prefaced by uniprot isoform id, ex P1234-3:2..4 ignored and TURN 10..14 ignored for NP_BIND split()
* parses 2 position range format ( .. between values) and ignores single position annotation
* outputs position range separated by -

In [4]:
nucleotide= 'TURN 10..14;  /note="FAD";  /evidence="ECO:0000250|UniProtKB:Q9HFE4"; TURN 41..41;  /note="FAD";  /evidence="ECO:0000250|UniProtKB:Q9HFE4"; NP_BIND 62..63;  /note="FAD";  /evidence="ECO:0000250|UniProtKB:Q9HFE4"; NP_BIND 196..199;  /note="NADP";  /evidence="ECO:0000250|UniProtKB:Q9HFE4"'
nucleotide = nucleotide.split("NP_BIND ")
nuc_coords=[]
for item in nucleotide:
    # check if its a range
    if ".." in item:
        # ignores non NP_BIND #..# in item that begins with NP_BIND by spliting on NP_BIND
        nuc_coords.append(item.partition(";  ")[0].split(".."))
nuc_coords2 = []
for item in nuc_coords:
    if len(item) > 1:
        # checks if there is isoform id preface or if range for other type of annotation
        if "-" not in item[0] and " " not in item[0]:
            nuc_coords2.append(item)
npbind_ranges = [x + "-" + y for x, y in nuc_coords2]


print(npbind_ranges)

['62-63', '196-199']


## parsing DNA BINDING, Position and type of a DNA-binding domain 
```python 
dna = 'DNA_BIND 17..21;  /evidence="ECO:0000250"'
dna = 'DNA_BIND 170..253;  /note="ETS";  /evidence="ECO:0000255|PROSITE-ProRule:PRU00237"'
dna = 'DNA_BIND 170..362'
```
**what parsing code does..**
* splits string on certain all caps word like NP_BIND, ZN_FING, STRAND, DOMAIN, etc.
* ignores positions prefaced by something other than all caps word used to split string and positions prefaced by uniprot isoform id, ex P1234-3:2..4 ignored and TURN 10..14 ignored for NP_BIND split()
* parses 2 position range format ( .. between values) and ignores single position annotation
* outputs position range separated by -

In [5]:
dna= 'DNA_BIND 170..53;  /note="ETS";  /evidence="ECO:0000255|PROSITE-ProRule:PRU00237"'
dna = dna.split("DNA_BIND ")
dna_coords=[]
for item in dna:
    # check if its a range
    if ".." in item:
        # ignores non DNA_BIND prefaced ranges in item by spliting on DNA_BIND
        dna_coords.append(item.partition(";  ")[0].split(".."))

dna_coords2 = []
for item in dna_coords:
    if len(item) > 1:
       # checks if there is isoform id preface or if range for other type of annotation
        if "-" not in item[0] and " " not in item[0]:
            dna_coords2.append(item)
dnabind_ranges = [x + "-" + y for x, y in dna_coords2]


print(dnabind_ranges)

['170-53']


## parsing ZINC BINDING, Position(s) and type(s) of zinc fingers within the protein 
```python 
zinc = 'ZN_FING 360..382;  /note="C2H2-type 1";  /evidence="ECO:0000255|PROSITE-ProRule:PRU00042"; ZN_FING 390..412;  /note="C2H2-type 2";  /evidence="ECO:0000255|PROSITE-ProRule:PRU00042"; ZN_FING 483..506;  /note="C2H2-type 3";  /evidence="ECO:0000255|PROSITE-ProRule:PRU00042"'
```
**what parsing code does..**
* splits string on certain all caps word like NP_BIND, ZN_FING, STRAND, DOMAIN, etc.
* ignores positions prefaced by something other than all caps word used to split string and positions prefaced by uniprot isoform id, ex P1234-3:2..4 ignored and TURN 10..14 ignored for NP_BIND split()
* parses 2 position range format ( .. between values) and ignores single position annotation
* outputs position range separated by -

In [112]:
zinc = 'ZNFING 360..382;  /note="C2H2-type 1";  /evidence="ECO:0000255|PROSITE-ProRule:PRU00042"; ZN_FING 390..412;  /note="C2H2-type 2";  /evidence="ECO:0000255|PROSITE-ProRule:PRU00042"; ZN_FING 483..506;  /note="C2H2-type 3";  /evidence="ECO:0000255|PROSITE-ProRule:PRU00042"'
zinc = zinc.split("ZN_FING ")
z_coords=[]
for item in zinc:
    # check if its a range
    if ".." in item:
        # ignores non ZN_FING prefaced ranges in item by spliting on ZN_FING
        z_coords.append(item.partition(";  ")[0].split(".."))
z_coords2 = []
for item in z_coords:
    if len(item) > 1:
        # checks if there is isoform id preface or if range for other type of annotation
        if "-" not in item[0] and " " not in item[0]:
            z_coords2.append(item)
zinc_ranges = [x + "-" + y for x, y in z_coords2]


print(zinc_ranges)

['390-412', '483-506']


## parsing MOTIF, Short (up to 20 amino acids) sequence motif of biological interest 
```python 
motif = 'MOTIF 996..999;  /note="Prevents secretion from ER";  /evidence="ECO:0000255"'
motif = 'MOTIF O60313-2:217..222;  /note="LQQQIQ motif";  /evidence="ECO:0000269|PubMed:16778770"; MOTIF O60313-9:181..186;  /note="LQQQIQ motif";  /evidence="ECO:0000269|PubMed:16778770"; MOTIF O60313-10:235..240;  /note="LQQQIQ motif";  /evidence="ECO:0000269|PubMed:16778770"'
```
**what parsing code does..**
* splits string on certain all caps word like NP_BIND, ZN_FING, STRAND, DOMAIN, etc.
* ignores positions prefaced by something other than all caps word used to split string and positions prefaced by uniprot isoform id, ex P1234-3:2..4 ignored and TURN 10..14 ignored for NP_BIND split()
* parses 2 position range format ( .. between values) and ignores single position annotation
* outputs position range separated by -

In [88]:
motif = 'MOTIF 111222;  /note="LQQQIQ motif";  /evidence="ECO:0000269|PubMed:16778770"; MOTIF 181..186;  /note="LQQQIQ motif";  /evidence="ECO:0000269|PubMed:16778770"; MOTIF O60313-10:235..240;  /note="LQQQIQ motif";  /evidence="ECO:0000269|PubMed:16778770"'
motif = motif.split("MOTIF ")
motif_coords=[]
for item in motif:
    # check if its a range
    if ".." in item:
        # ignores non MOTIF prefaced ranges in item by spliting on MOTIF
        motif_coords.append(item.partition(";  ")[0].split(".."))
motif_coords2 = []
for item in motif_coords:
    if len(item) > 1:
        # checks if there is isoform id preface or if range for other type of annotation
        if "-" not in item[0] and " " not in item[0]:
            # ignore ranges for non-canonical sequence since im only checking pos and seq identify of canonical 
            motif_coords2.append(item)
motif_ranges = [x + "-" + y for x, y in motif_coords2]


print(motif_ranges)

['181-186']


## parsing REGION, Region of interest in the sequence 
**for Disordered position(s)**
```python 
region= 'REGION 98..109;  /note="J region (By similarity to lambda light-chain)"; REGION 110..214;  /note="C region (By similarity to lambda light-chain)"; REGION 98..118;  /note="Disordered";  /evidence="ECO:0000256|SAM:MobiDB-lite"'
```
**what parsing code does..**
* splits string on certain all caps word like NP_BIND, ZN_FING, STRAND, DOMAIN, etc.
* ignores positions prefaced by something other than all caps word used to split string and positions prefaced by uniprot isoform id, ex P1234-3:2..4 ignored and TURN 10..14 ignored for NP_BIND split()
* only parses positions that are describing 'Disordered' feature
* parses range format (includes .. between values) and single position annotation
* single position annotation converted to range
* outputs 2 position range and single position as values separated by -

In [114]:
region= 'REGION 9109;  /note="Disordered (By similarity to lambda light-chain)"; REGION 110..214;  /note="C region (By similarity to lambda light-chain)"; REGION 98..118;  /note="Disordered";  /evidence="ECO:0000256|SAM:MobiDB-lite"'
disorder = region.split("REGION ")
disorder_coords=[]
for item in disorder:
    if "Disordered" in item:
        disorder_coords.append(item.partition(";  ")[0].split(".."))
disorder_coords2 = []
for item in disorder_coords:
    if len(item) > 1:
        # checks if there is isoform id preface or if range for other type of annotation
        if "-" not in item[0] and " " not in item[0]:
            disorder_coords2.append(item)
    elif len(item) == 1:
        # case of single position, make into range format
        # checks if there is isoform id preface or if range for other type of annotation
        if "-" not in item[0] and " " not in item[0]:
            v = []
            v.append(item[0])
            v.append(item[0])
            disorder_coords2.append(v)
    else:
        continue
disorder_ranges = [x + "-" + y for x, y in disorder_coords2]


print(disorder_ranges)

['9109-9109', '98-118']


## parsing DOMAIN, Position and type of each modular protein domain 
```python 
domain = 'DOMAIN 1..101;  /note="C2 1";  /evidence="ECO:0000255|PROSITE-ProRule:PRU00041"; DOMAIN 181..300;  /note="C2 2";  /evidence="ECO:0000255|PROSITE-ProRule:PRU00041"; DOMAIN 339..474;  /note="C2 3";  /evidence="ECO:0000255|PROSITE-ProRule:PRU00041"; DOMAIN 1123..1251;  /note="C2 4";  /evidence="ECO:0000255|PROSITE-ProRule:PRU00041"; DOMAIN 1282..1410;  /note="C2 5";  /evidence="ECO:0000255|PROSITE-ProRule:PRU00041"; DOMAIN 1536..1654;  /note="C2 6";  /evidence="ECO:0000255|PROSITE-ProRule:PRU00041"; DOMAIN 1772..1920;  /note="C2 7";  /evidence="ECO:0000255|PROSITE-ProRule:PRU00041"'
```
**what parsing code does..**
* splits string on certain all caps word like NP_BIND, ZN_FING, STRAND, DOMAIN, etc.
* ignores positions prefaced by something other than all caps word used to split string and positions prefaced by uniprot isoform id, ex P1234-3:2..4 ignored and TURN 10..14 ignored for NP_BIND split()
* parses 2 position range format ( .. between values) and ignores single position annotation
* outputs position range separated by -

In [117]:
domain = 'DOMAIN P123454-2:1..101;  /note="C2 1";  /evidence="ECO:0000255|PROSITE-ProRule:PRU00041"; DOMAIN 299..300;  /note="C2 2";  /evidence="ECO:0000255|PROSITE-ProRule:PRU00041"; DOMAIN 339..474;  /note="C2 3";  /evidence="ECO:0000255|PROSITE-ProRule:PRU00041"; DOMAIN 1123..1251;  /note="C2 4";  /evidence="ECO:0000255|PROSITE-ProRule:PRU00041"; DOMAIN 1282..1410;  /note="C2 5";  /evidence="ECO:0000255|PROSITE-ProRule:PRU00041"; DOMAIN 1536..1654;  /note="C2 6";  /evidence="ECO:0000255|PROSITE-ProRule:PRU00041"; DOMAIN 1772..1920;  /note="C2 7";  /evidence="ECO:0000255|PROSITE-ProRule:PRU00041"'
domain = domain.split("DOMAIN ")
domain_coords=[]
for item in domain:
    # check if its a range
    if ".." in item:
        # ignores non DOMAIN prefaced ranges in item by spliting on DOMAIN
        domain_coords.append(item.partition(";  ")[0].split(".."))
domain_coords2 = []
for item in domain_coords:
    if len(item) > 1:
        # checks if there is isoform id preface or if range for other type of annotation
        if "-" not in item[0] and " " not in item[0]:
            domain_coords2.append(item)
domain_ranges = [x + "-" + y for x, y in domain_coords2]


print(domain_ranges)

['299-300', '339-474', '1123-1251', '1282-1410', '1536-1654', '1772-1920']


## parsing BETA STRAND, Beta strand regions within the experimentally determined protein structure

```python 
beta = 'STRAND 1..3;  /evidence="ECO:0007829|PDB:6KVG"; STRAND 40..42;  /evidence="ECO:0007829|PDB:6KVG"; STRAND 210..212;  /evidence="ECO:0007829|PDB:6KVG"; STRAND 243..246;  /evidence="ECO:0007829|PDB:6KVG"'
```
**what parsing code does..**
* splits string on certain all caps word like NP_BIND, ZN_FING, STRAND, DOMAIN, etc.
* ignores positions prefaced by something other than all caps word used to split string and positions prefaced by uniprot isoform id, ex P1234-3:2..4 ignored and TURN 10..14 ignored for STRAND split()
* parses 2 position range format ( .. between values) and ignores single position annotation
* outputs position range separated by -

In [120]:
beta = 'STRAD 1..3;  /evidence="ECO:0007829|PDB:6KVG"; STRAND 40..42;  /evidence="ECO:0007829|PDB:6KVG"; STRAND 210..212;  /evidence="ECO:0007829|PDB:6KVG"; STRAND 243..246;  /evidence="ECO:0007829|PDB:6KVG"'
strand_coords = []
beta = beta.split("STRAND ")
for item in beta:
    # check if its a range
    if ".." in item:
        # ignores non STRAND #..# in item by spliting on STRAND
        strand_coords.append(item.partition(";  ")[0].split(".."))
strand_coords2 = []
for item in strand_coords:
    if len(item) > 1:
        # checks if there is isoform id preface or if range for other type of annotation
        if "-" not in item[0] and " " not in item[0]:
            strand_coords2.append(item)
strand_ranges = [x + "-" + y for x, y in strand_coords2]

print(strand_ranges)

['40-42', '210-212', '243-246']


## parsing HELIX, Helical regions within the experimentally determined protein structure 
```python 
helix = 'HELIX 89..92;  /evidence="ECO:0007829|PDB:4IQH"; HELIX 983..985;  /evidence="ECO:0007829|PDB:4CAH"'
```
**what parsing code does..**
* splits string on certain all caps word like NP_BIND, ZN_FING, STRAND, DOMAIN, etc.
* ignores positions prefaced by something other than all caps word used to split string and positions prefaced by uniprot isoform id, ex P1234-3:2..4 ignored and TURN 10..14 ignored for STRAND split()
* parses 2 position range format ( .. between values) and ignores single position annotation
* outputs position range separated by -

In [101]:
helix = 'HELIX 89..92;  /evidence="ECO:0007829|PDB:4IQH"; HELIX 983..985;  /evidence="ECO:0007829|PDB:4CAH"'
helix_coords = []
helix = helix.split("HELIX ")
for item in helix:
    # check if its a range
    if ".." in item:
        # ignores non HELIX #..# in item by spliting on HELIX
        helix_coords.append(item.partition(";  ")[0].split(".."))
helix_coords2 = []
for item in helix_coords:
    if len(item) > 1:
        # checks if there is isoform id preface or if range for other type of annotation
        if "-" not in item[0] and " " not in item[0]:
            helix_coords2.append(item)
helix_ranges = [x + "-" + y for x, y in helix_coords2]

print(helix_ranges)

['89-92', '983-985']


## parsing TURN, Turns within the experimentally determined protein structure 
```python 
turn = 'TURN 99..101;  /evidence="ECO:0007829|PDB:1QGR"; TURN 202..205;  /evidence="ECO:0007829|PDB:1IBR"'
```
**what parsing code does..**
* splits string on certain all caps word like NP_BIND, ZN_FING, STRAND, DOMAIN, etc.
* ignores positions prefaced by something other than all caps word used to split string and positions prefaced by uniprot isoform id, ex P1234-3:2..4 ignored and TURN 10..14 ignored for STRAND split()
* parses 2 position range format ( .. between values) and ignores single position annotation
* outputs position range separated by -

In [104]:
turn = 'TRN 99..99;  /evidence="ECO:0007829|PDB:1QGR"; TURN 202..205;  /evidence="ECO:0007829|PDB:1IBR"'
turn_coords = []
turn = turn.split("TURN ")
for item in turn:
    # check if its a range
    if ".." in item:
        # ignores non TURN #..# in item by spliting on TURN
        turn_coords.append(item.partition(";  ")[0].split(".."))
turn_coords2 = []
for item in turn_coords:
    if len(item) > 1:
        # checks if there is isoform id preface or if range for other type of annotation
        if "-" not in item[0] and " " not in item[0]:
            turn_coords2.append(item)
turn_ranges = [x + "-" + y for x, y in turn_coords2]

print(turn_ranges)

['202-205']
